Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-with-automated-machine-learning-step.png)

# Azure Machine Learning Pipeline with AutoMLStep (Udacity Course 2)
This notebook demonstrates the use of AutoMLStep in Azure Machine Learning Pipeline.

## Introduction
In this example we showcase how you can use AzureML Dataset to load data for AutoML via AML Pipeline. 

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you have executed the [configuration](https://aka.ms/pl-config) before running this notebook.

In this notebook you will learn how to:
1. Create an `Experiment` in an existing `Workspace`.
2. Create or Attach existing AmlCompute to a workspace.
3. Define data loading in a `TabularDataset`.
4. Configure AutoML using `AutoMLConfig`.
5. Use AutoMLStep
6. Train the model using AmlCompute
7. Explore the results.
8. Test the best fitted model.

## Azure Machine Learning and Pipeline SDK-specific imports

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.38.0


## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-188528
aml-quickstarts-188528
southcentralus
48a74bb7-9950-4cc1-9caa-5d50f995cc55


## Create an Azure ML experiment
Let's create an experiment named "automlstep-classification" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

*Udacity Note:* There is no need to create an Azure ML experiment, this needs to re-use the experiment that was already created


In [3]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'bank-marketing'
project_folder = './pipeline-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
bank-marketing,quick-starts-ws-188528,Link to Azure Machine Learning studio,Link to Documentation


### Create or Attach an AmlCompute cluster
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run. In this tutorial, you get the default `AmlCompute` as your training compute resource.

**Udacity Note** There is no need to create a new compute target, it can re-use the previous cluster

In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "project02"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Data

**Udacity note:** Make sure the `key` is the same name as the dataset that is uploaded, and that the description matches. If it is hard to find or unknown, loop over the `ws.datasets.keys()` and `print()` them.
If it *isn't* found because it was deleted, it can be recreated with the link that has the CSV 

In [5]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "bank marketing"
description_text = "Bank marketing DataSet for Udacity Course 2"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
#df.describe()

### Review the Dataset Result

You can peek the result of a TabularDataset at any range using `skip(i)` and `take(j).to_pandas_dataframe()`. Doing so evaluates only `j` records for all the steps in the TabularDataset, which makes it fast even against large datasets.

`TabularDataset` objects are composed of a list of transformation steps (optional).

In [6]:
dataset.take(5).to_pandas_dataframe()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


## Train
This creates a general AutoML settings object.
**Udacity notes:** These inputs must match what was used when training in the portal. `time_column_name` has to be `cnt` for example.

In [7]:
automl_settings = {
    "experiment_timeout_minutes": 16,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'AUC_weighted',
    "n_cross_validations": 5
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="y",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

#### Create Pipeline and AutoMLStep

You can define outputs for the AutoMLStep using TrainingOutput.

In [8]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

Create an AutoMLStep.

In [11]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [12]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

In [13]:
pipeline_run = experiment.submit(pipeline)

Created step automl_module [d6795484][3bac8f45-095c-4342-81f2-68f98f5242bf], (This step will run and generate new outputs)
Submitted PipelineRun ed28b850-62f8-4eb5-89a9-d1198a741d21
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/ed28b850-62f8-4eb5-89a9-d1198a741d21?wsid=/subscriptions/48a74bb7-9950-4cc1-9caa-5d50f995cc55/resourcegroups/aml-quickstarts-188528/workspaces/quick-starts-ws-188528&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254


In [14]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [15]:
pipeline_run.wait_for_completion()

PipelineRunId: ed28b850-62f8-4eb5-89a9-d1198a741d21
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/ed28b850-62f8-4eb5-89a9-d1198a741d21?wsid=/subscriptions/48a74bb7-9950-4cc1-9caa-5d50f995cc55/resourcegroups/aml-quickstarts-188528/workspaces/quick-starts-ws-188528&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
PipelineRun Status: Running


StepRunId: 1b59964a-36b0-4621-b2c9-4f1339ea884a
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/1b59964a-36b0-4621-b2c9-4f1339ea884a?wsid=/subscriptions/48a74bb7-9950-4cc1-9caa-5d50f995cc55/resourcegroups/aml-quickstarts-188528/workspaces/quick-starts-ws-188528&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
StepRun( automl_module ) Status: Running

StepRun(automl_module) Execution Summary
StepRun( automl_module ) Status: Finished

Warnings:
No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/py

'Finished'

## Examine Results

### Retrieve the metrics of all child runs
Outputs of above run can be used as inputs of other steps in pipeline. In this tutorial, we will examine the outputs by retrieve output data and running some tests.

In [16]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

Downloaded azureml/1b59964a-36b0-4621-b2c9-4f1339ea884a/metrics_data, 1 files out of an estimated total of 1


In [17]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

,1b59964a-36b0-4621-b2c9-4f1339ea884a_11,1b59964a-36b0-4621-b2c9-4f1339ea884a_12,1b59964a-36b0-4621-b2c9-4f1339ea884a_13,1b59964a-36b0-4621-b2c9-4f1339ea884a_25,1b59964a-36b0-4621-b2c9-4f1339ea884a_15,1b59964a-36b0-4621-b2c9-4f1339ea884a_28,1b59964a-36b0-4621-b2c9-4f1339ea884a_31,1b59964a-36b0-4621-b2c9-4f1339ea884a_24,1b59964a-36b0-4621-b2c9-4f1339ea884a_26,1b59964a-36b0-4621-b2c9-4f1339ea884a_33,...,1b59964a-36b0-4621-b2c9-4f1339ea884a_19,1b59964a-36b0-4621-b2c9-4f1339ea884a_17,1b59964a-36b0-4621-b2c9-4f1339ea884a_21,1b59964a-36b0-4621-b2c9-4f1339ea884a_14,1b59964a-36b0-4621-b2c9-4f1339ea884a_22,1b59964a-36b0-4621-b2c9-4f1339ea884a_20,1b59964a-36b0-4621-b2c9-4f1339ea884a_23,1b59964a-36b0-4621-b2c9-4f1339ea884a_18,1b59964a-36b0-4621-b2c9-4f1339ea884a_16,1b59964a-36b0-4621-b2c9-4f1339ea884a_27
accuracy,[0.7655538694992412],[0.9150834597875569],[0.8881942336874052],[0.9125644916540212],[0.9055235204855843],[0.9161456752655539],[0.9133232169954477],[0.9146585735963582],[0.9093778452200304],[0.9136267071320182],...,[0.8879817905918058],[0.8176631259484066],[0.9147192716236724],[0.9115326251896813],[0.9162063732928679],[0.908619119878604],[0.7868892261001517],[0.9165705614567526],[0.8620030349013657],[0.9084066767830047]
recall_score_macro,[0.7346101493216579],[0.7431206096810072],[0.5013142000586454],[0.704074496211164],[0.6031664313077449],[0.7484046856101216],[0.6897754749553021],[0.7194306306840508],[0.6940305774161666],[0.750684533747165],...,[0.500133155792277],[0.8519636306950092],[0.747306337390383],[0.6934594904149399],[0.7509024402711638],[0.6888796054799327],[0.8542643265250736],[0.7571767517842689],[0.8703876343893802],[0.677882543477472]
log_loss,[0.5461579007813043],[0.2014204930088901],[0.28908986259019226],[0.18771881377256389],[0.20840659157343494],[0.17445330019132363],[0.1849743854240511],[0.31924620227511874],[0.21029067950559552],[0.1752132173824807],...,[0.28781664747303826],[0.39259459217530673],[0.31394291237534866],[0.2066352008600357],[0.17365288433672377],[0.21221930933730118],[0.41123930449655566],[0.3136595435425747],[0.37126166500049623],[0.2124044434788533]
norm_macro_recall,[0.4692202986433157],[0.48624121936201464],[0.002628400117290708],[0.40814899242232805],[0.20633286261548975],[0.49680937122024316],[0.3795509499106041],[0.4388612613681014],[0.3880611548323331],[0.5013690674943299],...,[0.00026631158455394657],[0.7039272613900187],[0.49461267478076587],[0.38691898082987974],[0.5018048805423276],[0.3777592109598655],[0.7085286530501469],[0.5143535035685377],[0.7407752687787602],[0.35576508695494413]
f1_score_micro,[0.7655538694992412],[0.9150834597875569],[0.8881942336874051],[0.9125644916540212],[0.9055235204855843],[0.9161456752655539],[0.9133232169954477],[0.9146585735963582],[0.9093778452200304],[0.9136267071320182],...,[0.8879817905918058],[0.8176631259484066],[0.9147192716236722],[0.9115326251896813],[0.9162063732928679],[0.908619119878604],[0.7868892261001517],[0.9165705614567526],[0.8620030349013657],[0.9084066767830047]
average_precision_score_macro,[0.7107336809275498],[0.8246785192865105],[0.7020727815247858],[0.8169619093890272],[0.827432923293963],[0.8279709527878356],[0.8264679008475486],[0.8200936562444177],[0.7912816131894216],[0.8226010613711325],...,[0.7100467928475759],[0.7741023159428455],[0.817462566074072],[0.8099295855607158],[0.8287254202105364],[0.7885704110589759],[0.7631712509937331],[0.8195168702681755],[0.7849125105089675],[0.7886553211225649]
f1_score_weighted,[0.8033586365450294],[0.9108885614124516],[0.8359591827594391],[0.9042741388692839],[0.8804258246711463],[0.9123053594246325],[0.9027810938282433],[0.9076330865902251],[0.9004984952956707],[0.9105811014951801],...,[0.8353272409172601],[0.8465350004271676],[0.9110801372656931],[0.9019566250314902],[0.9126158203977524],[0.899206149781534],[0.8235984718580586],[0.9135391586516155],[0.880302084359793],[0.8973793422461844]
precision_score_weighted,[0.8772690141847928],[0.9087650484115178],[0.8259058857816536],[0.902

### Retrieve the Best Model

In [18]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

Downloaded azureml/1b59964a-36b0-4621-b2c9-4f1339ea884a/model_data, 1 files out of an estimated total of 1


In [27]:
import azureml.train.automl

In [19]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mn...
    gpu_training_param_dict={'processing_unit_type': 'cpu'}
), random_state=None, reg_alpha=0.3684210526315789, reg_lambda=1, subsample=1))], verbose=False))], flatten_transform=None, weights=[0.2857142857142857, 0.07142857142857142, 0.14285714285714285, 0.07142857142857142, 0.14285714285714285, 0.07142857142857142, 0.21428571428571427]))],
                                       'verbose': False},
                             y_transformer={},
                             y_transformer_name='LabelEncoder')

In [20]:
best_model.steps

[('datatransformer',
  DataTransformer(
      task='classification',
      is_onnx_compatible=False,
      enable_feature_sweeping=True,
      enable_dnn=False,
      force_text_dnn=False,
      feature_sweeping_timeout=86400,
      featurization_config=None,
      is_cross_validation=True,
      feature_sweeping_config={}
  )),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(
      estimators=[('0', Pipeline(
          memory=None,
          steps=[('maxabsscaler', MaxAbsScaler(
              copy=True
          )), ('lightgbmclassifier', LightGBMClassifier(
              min_data_in_leaf=20,
              random_state=None,
              n_jobs=1,
              problem_info=ProblemInfo(
                  gpu_training_param_dict={'processing_unit_type': 'cpu'}
              )
          ))],
          verbose=False
      )), ('22', Pipeline(
          memory=None,
          steps=[('standardscalerwrapper', StandardScalerWrapper(
              copy=True,
             

## Publish and run from REST endpoint

Run the following code to publish the pipeline to your workspace. In your workspace in the portal, you can see metadata for the pipeline including run history and durations. You can also run the pipeline manually from the portal.

Additionally, publishing the pipeline enables a REST endpoint to rerun the pipeline from any HTTP library on any platform.


In [21]:
# Ensure that the workspace is once again available
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-188528
aml-quickstarts-188528
southcentralus
48a74bb7-9950-4cc1-9caa-5d50f995cc55


In [22]:
# NOTE: update these to match your existing experiment name and a previous experiment
experiment_name = 'bank-marketing'
project_folder = './pipeline-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
bank-marketing,quick-starts-ws-188528,Link to Azure Machine Learning studio,Link to Documentation


In [23]:
from azureml.pipeline.core import PipelineRun

run_id = "ed28b850-62f8-4eb5-89a9-d1198a741d21"
pipeline_run = PipelineRun(experiment, run_id)

In [24]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Bank Marketing", description="Training bank marketing pipeline", version="1.0")

published_pipeline


Name,Id,Status,Endpoint
Bank Marketing,c91bf83d-1d02-4518-a8c0-6a145caa9c3b,Active,REST Endpoint


Authenticate once again, to retrieve the `auth_header` so that the endpoint can be used

In [25]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()



Get the REST url from the endpoint property of the published pipeline object. You can also find the REST url in your workspace in the portal. Build an HTTP POST request to the endpoint, specifying your authentication header. Additionally, add a JSON payload object with the experiment name and the batch size parameter. As a reminder, the process_count_per_node is passed through to ParallelRunStep because you defined it is defined as a PipelineParameter object in the step configuration.

Make the request to trigger the run. Access the Id key from the response dict to get the value of the run id.


In [26]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-bank-rest-endpoint"}
                        )

In [27]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

Submitted pipeline run:  cdf360b6-8fb1-47ca-bce7-c7721b2a1b83


Use the run id to monitor the status of the new run. This will take another 10-15 min to run and will look similar to the previous pipeline run, so if you don't need to see another pipeline run, you can skip watching the full output.

In [28]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["pipeline-bank-rest-endpoint"], run_id)
RunDetails(published_pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …